# Datenaufbereitung RCP2.6 Niederschlag

###  1. Packete laden, Pfade definieren und Daten laden

In [4]:
# Benötigte Packete
import xarray as xr
import geopandas as gpd
import rioxarray
import numpy as np
import pandas as pd
from pyproj import Geod
from shapely.geometry import Point
import os
from glob import glob

# Dateipfade definieren
nc_dir = "https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/rcp26pr/raw/"

#Shapefile Pfad
shp_path= "https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/Murgtal_boundingbox/Murgtal_boundingbox.shp"

#Hier muss der lokale Pfad definiert werden:
output_csv = "hier-lokalen-Pfad-definieren/1_daten/rcp26pr/murg_pr_rcp26_mean.csv"

# Laden der Shapefile für das Murgeinzugsgebiet, Shapefile in Koordinatensystem EPSG 4326 umwandeln
murg_shape = gpd.read_file(shp_path).to_crs("EPSG:4326")

#Liste erstellen für Tageswerte
all_data = []

# ALle Dateien holen sortiert nach Jahr
nc_files = sorted(glob(os.path.join(nc_dir, "*.nc")))


### 2. Loop zur Datenaufbereitung aller .nc Daten
Eine Shapefile kann nicht direkt auf eine .nc Datei angewendet werden. Shapefile = Vektoren, NC-Datei = Gitterzellen --> Maske schaut welche Gitterpunkte betroffen sind, sie hat das gleiche Format
wie die Klimadaten in der .nc Datei

In [2]:
# Dateien durchlaufen
for nc_path in nc_files:
    print("Verarbeite:", nc_path)
    
    # Öffnen
    pr_ds = xr.open_dataset(nc_path)
    pr = pr_ds["pr"]  # Niederschlagsvariable

    # 2D lat/lon-Arrays aus Klimamodelldaten nehmen
    lats = pr_ds["lat"].values
    lons = pr_ds["lon"].values

    # Maske erstellen für alle Gitterzellen, welche innerhalb des Murggebiets liegen
    mask = np.zeros_like(lats, dtype=bool) 
    for geom in murg_shape.geometry: 
        mask |= np.vectorize(lambda x, y: geom.contains(Point(x, y)))(lons, lats) #Gitterzellen die in geom liegen werden auf TRUE gesetzt --> Maske erstellt
    mask_da = xr.DataArray(mask, dims=("rlat", "rlon")) # Maske als DataArray umwandeln, damit es mit den Klimamodelldaten übereinstimmt


    # Nur Werte des Klimamodells innerhalb des Murggebiets behalten
    pr_murg = pr.where(mask_da)

    mean_mmday = pr_murg.mean(dim=["rlat", "rlon"]) * 86400 # in Grad

    df = pd.DataFrame({
    "date": mean_mmday["time"].values,
    "precip_mm_day": mean_mmday.values
    })

    # an Liste hängen
    all_data.append(df)


Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20060101-20101231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20110101-20151231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20160101-20201231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20210101-20251231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20260101-20301231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20310101-20351231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20360101-20401231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20410101-20451231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_PIK-STARS3_v1-r3_day_20460101-20501231.nc
Verarbeite: data/rcp26pr/pr_EUR-11_MP

### 3. Erstellen einer CSV mit allen Niederschlagsdaten von 2006 bis 2100

In [3]:
# Alle Daten zusammenfügen
final_df = pd.concat(all_data, ignore_index=True)

# Als csv speichern
final_df.to_csv(output_csv, index=False)
print("Gesamte CSV gespeichert:", output_csv)

Gesamte CSV gespeichert: data/murg_pr_rcp26_mean.csv


In [5]:
print(df["precip_mm_day"].describe())

count    1826.000000
mean        3.681243
std         7.469471
min         0.000000
25%         0.000000
50%         0.125000
75%         4.079166
max        95.450005
Name: pr_mm_tag, dtype: float64
